# Anomaly Detection using Time Series Insights API (Basics)

In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
!pip3 install --upgrade oauth2client 

In [2]:
# Load libraries

from oauth2client.client import GoogleCredentials
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
import json
import requests

In [3]:
pd.set_option('display.max_colwidth', None)
client = bigquery.Client()
PROJECT_ID="<your-project-id>"

In [6]:
# Read raw data and format 

sql = """
 select FARM_FINGERPRINT(CONCAT(time, temp, Humidity, Light, h2_raw)) groupId, 
            FORMAT_TIMESTAMP("%Y-%m-%dT%X%Ez", time, "UTC") eventTime, 
            temp, 
            Humidity, 
            Light, 
            h2_raw,
            'LTHH' as measure
        from `<project-id>.<dataset-id>.full_ts_data`
"""
df = client.query(sql).to_dataframe()
df = df.melt(id_vars=['groupId','eventTime'])
df.head()

,groupId,eventTime,variable,value
0,-2371371181525175581,2021-06-25T21:33:44+00:00,temp,55.85
1,-1611801076400518757,2021-06-25T21:37:13+00:00,temp,55.85
2,-1786144178866771453,2021-06-25T21:37:36+00:00,temp,55.83
3,-3830910545382395153,2021-06-25T21:38:00+00:00,temp,55.85
4,6305421944528799476,2021-06-25T21:38:24+00:00,temp,55.85


In [7]:
df.shape

(1011335, 4)

In [ ]:
# Write melt df to bq table --DO NOT RUN THIS (TABLE ALREADY CREATED IN BQ)
table_id="<table-id>"
pandas_gbq.to_gbq(df,table_id,project_id=PROJECT_ID)

In [ ]:
# OPTIONAL - DO NOT RUN (Read full json and convert into new line delimited json)

with open("/home/jupyter/jbl-full-ts.json", "r") as read_file:
    data = json.load(read_file)
result = [json.dumps(record) for record in data]
with open('nd-proceesed-full.json', 'w') as obj:
    for i in result:
        obj.write(i+'\n')

In [8]:
# Read melted data from BQ table and prepare json for time series indight API dataset

sql_out = """
with data as
    (
        select groupId, eventTime, STRUCT(variable as name, value as doubleVal) as dimensions 
        from (
                select * from `<project-id>.<dataset-id>.anomaly_data`
                order by eventTime, variable
             )
    )

    SELECT eventTime, groupId, ARRAY_AGG(dimensions) AS dimensions FROM data GROUP BY eventTime, groupId 
"""
df_out = client.query(sql_out).to_dataframe()
df_out.head()

,eventTime,groupId,dimensions
0,2021-06-14T00:00:04+00:00,2583958225776393023,"[{'name': 'Humidity', 'doubleVal': 36.0}, {'name': 'Light', 'doubleVal': 99.0}, {'name': 'h2_raw', 'doubleVal': 1041.0}, {'name': 'temp', 'doubleVal': 36.97}]"
1,2021-06-14T00:00:28+00:00,941253816505829577,"[{'name': 'Humidity', 'doubleVal': 36.0}, {'name': 'Light', 'doubleVal': 102.0}, {'name': 'h2_raw', 'doubleVal': 1021.0}, {'name': 'temp', 'doubleVal': 36.93}]"
2,2021-06-14T00:00:51+00:00,4798976411894260670,"[{'name': 'Humidity', 'doubleVal': 36.0}, {'name': 'Light', 'doubleVal': 102.0}, {'name': 'h2_raw', 'doubleVal': 1051.0}, {'name': 'temp', 'doubleVal': 36.97}]"
3,2021-06-14T00:01:14+00:00,2061911259308007249,"[{'name': 'Humidity', 'doubleVal': 36.0}, {'name': 'Light', 'doubleVal': 100.0}, {'name': 'h2_raw', 'doubleVal': 1040.0}, {'name': 'temp', 'doubleVal': 36.95}]"
4,2021-06-14T00:01:38+00:00,7896826658989699415,"[{'name': 'Humidity', 'doubleVal': 36.0}, {'name': 'Light', 'doubleVal': 99.0}, {'name': 'h2_raw', 'doubleVal': 1040.0}, {'name': 'temp', 'doubleVal': 36.96}]"


In [9]:
# write data to json
df_out.to_json("files/df_converted.json", orient="records", lines=True)

# Interacting with Time series API

### Helper functions

In [4]:
# reads json file and returns request body

def read_json_file(path):
    with open(path) as json_file:
        query = json.load(json_file)
        
    return query

In [5]:
# TODO - REPLACE WITH SA ACCOUNT KEY
KEY_FILE = '/home/jupyter/files/svc-acc.json' #REPLACE THIS WITH PROJECT SPECIFIC SERVICE ACCOUNT KEY
ts_endpoint =  f'https://timeseriesinsights.googleapis.com/v1/projects/{PROJECT_ID}/datasets'

In [ ]:
# Loading credentials into auth_token - this is temp token that needs to be refreshed
token=GoogleCredentials.get_application_default().get_access_token().access_token
token 

In [7]:
# Function to interact with time series API

def query_ts(method, endpoint, data, auth_token):
    data = str(data)
    headers = {'Content-type': 'application/json', "Authorization": f"Bearer {auth_token}"}
    
    if method == "GET":
        resp = requests.get(endpoint, headers=headers)
    if method == "POST":
        resp = requests.post(endpoint, data=data, headers=headers)
    if method == "DELETE":
        resp = requests.delete(endpoint, headers=headers)
    
    return(resp.json())

### 1. Create "*create.json*" with dataset properties and dataset

In [ ]:
%%writefile client-files/tsi/create.json

{
    name: "jbl_timeseries_full",
    dataNames: [
        "measure",
        "Humidity",
        "Light",
        "h2_raw",
        "temp",
    ],
    dataSources: [
        {uri: "gs://demo-ts-data/jbl-full-ts.json"}
    ]
}

In [ ]:
with open('client-files/tsi/create.json', 'r') as myfile:
    data=myfile.read()
    
print(data)

In [ ]:
# DO NOT RUN - THIS WILL CREATE NEW DATASET
# Create dataset 
res = query_ts(method="POST", endpoint=ts_endpoint, data=data, auth_token=token)
res

### 2. List datasets

In [ ]:
res = query_ts(method="GET", endpoint=ts_endpoint, data=None, auth_token=token)
res

### 3. Create *query.json* with params and get forecast

#### Using light as *metric*

In [ ]:
%%writefile files/tsi/query.json

{
    detectionTime: "2021-07-06T17:00:00Z",
    slicingParams: {
        "dimensionNames": ["measure"]
        },
    timeseriesParams: {
        "forecastHistory": "86400s",
        "granularity": "3600s",
        "metric": "Light"
        },
   
    returnNonAnomalies: true,
    returnTimeseries: true
} 

#### Using temp as *metric*

In [ ]:
%%writefile files/tsi/query.json

{
    detectionTime: "2021-07-05T15:24:00Z",
    slicingParams: {
        "dimensionNames": ["measure"]
        },
    timeseriesParams: {
        "forecastHistory": "86400s",
        "granularity": "10s",
        "metric": "temp"
        },
   
    returnNonAnomalies: false,
    returnTimeseries: true
} 

In [38]:
with open('files/tsi/query.json', 'r') as myfile:
    query=myfile.read()
    
print(query)


{
    detectionTime: "2021-07-06T17:00:00Z",
    slicingParams: {
        "dimensionNames": ["measure"]
        },
    timeseriesParams: {
        "forecastHistory": "86400s",
        "granularity": "3600s",
        "metric": "Light"
        },
   
    returnNonAnomalies: true,
    returnTimeseries: true
} 



### 4. Fetch Timeseries

In [ ]:
%%writefile files/tsi/fetch.json

{
    "sliceDimensions": [
        {
            "dimension":"measure",
            "stringVal":"LTTH"
        }
        ],
      "timeInterval": {
        startTime: "2021-07-06T00:00:00Z",
        length: "172800s"
      },
      "granularity": "3600s",
      "metric": "Light"
}

In [ ]:
%%writefile files/tsi/fetch.json

{
    "sliceDimensions": [
        {
            "dimension":"measure",
            "stringVal":"LTTH"
        }
        ],
      "timeInterval": {
        startTime: "2021-07-05T00:24:00Z",
        length: "86400s"
      },
       "granularity": "60s",
       "metric": "temp"
}


In [24]:
with open('files/tsi/fetch.json', 'r') as myfile:
    fetch_data=myfile.read()
    
print(fetch_data)


{
    "sliceDimensions": [
        {
            "dimension":"measure",
            "stringVal":"LTTH"
        }
        ],
      "timeInterval": {
        startTime: "2021-07-05T00:24:00Z",
        length: "86400s"
      },
       "granularity": "60s",
       "metric": "temp"
}



In [ ]:
# Fetch timeseries for inspection

fetch_ds_endpt = f'https://timeseriesinsights.googleapis.com/v1/projects/{PROJECT_ID}/datasets/{dataset_name}:fetchTimeseries'
res = query_ts(method="POST", endpoint=fetch_ds_endpt, data=fetch_data, auth_token=token)
res

### 5. Delete timeseries

In [ ]:
# delete datasets

dataset_name = "test"
delete_ds_endpt = f'https://timeseriesinsights.googleapis.com/v1/projects/{PROJECT_ID}/datasets/{dataset_name}'
res = query_ts(method="DELETE", endpoint=delete_ds_endpt, data=None, auth_token=token)
res